## Import all libraries and Dataset

In [ ]:
import os
import numpy as np
import pandas as pd 
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.gridspec as gridspec
import shap

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, root_mean_squared_error
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
import joblib

: 

In [ ]:
# Import the Data
data= pd.read_excel('data.xlsx')
data.head()

## Preprocessing

In [ ]:
data.columns = [c.strip().lower() for c in data.columns]
data

In [ ]:
data['datetime'] = pd.to_datetime(data['datetime'], errors='coerce')
data

In [ ]:
#Convert each feeders from WH to MWH
for col in data.columns[1:]:  # Start from the second column
    data[col] = pd.to_numeric(data[col], errors='coerce') 
    data[col] = data[col] / 1e6
        
data

In [ ]:
# Check for missing values
data.isnull().sum()

In [ ]:
# Use Linear interpolation to fill missing values
data_clean = data.copy()
data_clean = data_clean.set_index('datetime')
# Interpolate missing values
data_clean = data_clean.interpolate(method='time')
data_clean = data_clean.reset_index()
# Check remaining missing values
data_clean.isnull().sum()

In [ ]:
data_clean.info()

In [ ]:
data_clean.describe()

## Feature Engineering

In [ ]:
data_clean = data_clean.copy()
data_clean['hour'] = data_clean['datetime'].dt.hour
data_clean['day_of_week'] = data_clean['datetime'].dt.dayofweek  # 0=Monday, 6=Sunday
data_clean['month'] = data_clean['datetime'].dt.month
data_clean['day'] = data_clean['datetime'].dt.day
data_clean['year'] = data_clean['datetime'].dt.year
data_clean['is_weekend'] = (data_clean['day_of_week'] >= 5).astype(int)

data_clean

In [ ]:
# 2. Create cyclical features for time variables (preserves cyclical nature)
data_clean['hour_sin'] = np.sin(2 * np.pi * data_clean['hour']/24)
data_clean['hour_cos'] = np.cos(2 * np.pi * data_clean['hour']/24)
data_clean['month_sin'] = np.sin(2 * np.pi * data_clean['month']/12)
data_clean['month_cos'] = np.cos(2 * np.pi * data_clean['month']/12)
data_clean['day_of_week_sin'] = np.sin(2 * np.pi * data_clean['day_of_week']/7)
data_clean['day_of_week_cos'] = np.cos(2 * np.pi * data_clean['day_of_week']/7)

data_clean.isnull().sum()


In [ ]:
# Create a list of all columns
all_columns = data_clean.columns.tolist()

# Identify feeder columns (all numeric columns except the engineered features)
time_feature_cols = ['hour', 'day_of_week', 'month', 'day', 'year', 'is_weekend', 
                     'hour_sin', 'hour_cos', 'month_sin', 'month_cos', 
                     'day_of_week_sin', 'day_of_week_cos']

# First column is datetime, the rest are feeder columns initially
feeder_cols = [col for col in all_columns if col not in ['datetime'] + time_feature_cols]

# Create new column order: datetime, time features, then feeders
new_column_order = ['datetime'] + time_feature_cols + feeder_cols

# Reorder the DataFrame
data_clean = data_clean[new_column_order]

# Verify the new order
data_clean

## Exploratory Data Analysis

In [ ]:
# Create a directory for storing plots if it doesn't exist
plot_dir = 'energy_plots'
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)

# Create a copy for plotting
plot_data = data_clean.copy()

# 1. Daily consumption patterns by hour
plt.figure(figsize=(10, 6))
hourly_consumption = plot_data.groupby('hour')[feeder_cols].mean()
hourly_consumption.mean(axis=1).plot()
plt.title('Average Hourly Energy Consumption Pattern')
plt.xlabel('Hour of Day')
plt.ylabel('Average Energy (MWH)')
plt.xticks(range(0, 24, 2))
plt.tight_layout()
plt.savefig(os.path.join(plot_dir, 'hourly_consumption.png'), dpi=300)
plt.show()

# 2. Weekly patterns by day
plt.figure(figsize=(10, 6))
daily_consumption = plot_data.groupby('day_of_week')[feeder_cols].mean()
daily_consumption.mean(axis=1).plot()
plt.title('Average Daily Energy Consumption Pattern')
plt.xlabel('Day of Week')
plt.ylabel('Average Energy (MWH)')
plt.xticks(range(7), ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])
plt.tight_layout()
plt.savefig(os.path.join(plot_dir, 'daily_consumption.png'), dpi=300)
plt.show()

# 3. Monthly seasonal patterns
plt.figure(figsize=(10, 6))
monthly_consumption = plot_data.groupby('month')[feeder_cols].mean()
monthly_consumption.mean(axis=1).plot()
plt.title('Monthly Energy Consumption Pattern')
plt.xlabel('Month')
plt.ylabel('Average Energy (MWH)')
plt.xticks(range(1, 13), ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
plt.tight_layout()
plt.savefig(os.path.join(plot_dir, 'monthly_consumption.png'), dpi=300)
plt.show()

# 4. Weekday vs Weekend comparison
# plt.figure(figsize=(10, 6))
# weekend_comparison = plot_data.groupby(['is_weekend', 'hour'])[feeder_cols].mean().mean(axis=1).unstack()
# weekend_comparison.plot()
# plt.title('Weekday vs Weekend Energy Consumption')
# plt.xlabel('Hour of Day')
# plt.ylabel('Average Energy (MWH)')
# plt.legend(['Weekday', 'Weekend'])
# plt.xticks(range(0, 24, 2))
# plt.tight_layout()
# plt.savefig(os.path.join(plot_dir, 'weekend_comparison.png'), dpi=300)
# plt.show()

# 5. Correlation heatmap between feeders
plt.figure(figsize=(12, 10))
correlation = plot_data[feeder_cols].corr()
sns.heatmap(correlation, annot=True, cmap='coolwarm', vmin=-1, vmax=1, fmt='.2f', linewidths=0.5)
plt.title('Correlation Between Energy Feeders')
plt.tight_layout()
plt.savefig(os.path.join(plot_dir, 'feeder_correlation.png'), dpi=300)
plt.show()

# 6. Time series plot of total consumption
plt.figure(figsize=(12, 6))
# Sample the data to avoid overcrowding (e.g., daily averages)
daily_data = plot_data.set_index('datetime').resample('D').mean()
total_energy = daily_data[feeder_cols].sum(axis=1)
total_energy.plot()
plt.title('Total Energy Consumption Over Time')
plt.xlabel('Date')
plt.ylabel('Total Energy (MWH)')
plt.tight_layout()
plt.savefig(os.path.join(plot_dir, 'total_consumption_trend.png'), dpi=300)
plt.show()

# 7. Boxplot of hourly consumption patterns
plt.figure(figsize=(15, 8))
hourly_data = plot_data.melt(id_vars=['hour'], 
                          value_vars=feeder_cols,
                          var_name='Feeder', 
                          value_name='Energy')
sns.boxplot(x='hour', y='Energy', data=hourly_data)
plt.title('Distribution of Energy Consumption by Hour')
plt.xlabel('Hour of Day')
plt.ylabel('Energy (MWH)')
plt.xticks(range(0, 24))
plt.tight_layout()
plt.savefig(os.path.join(plot_dir, 'hourly_distribution.png'), dpi=300)
plt.show()

# 8. Top consumers comparison
plt.figure(figsize=(12, 8))
feeder_means = plot_data[feeder_cols].mean().sort_values(ascending=False)
feeder_means.plot(kind='bar')
plt.title('Average Energy Consumption by Feeder')
plt.xlabel('Feeder')
plt.ylabel('Average Energy (MWH)')
plt.tight_layout()
plt.savefig(os.path.join(plot_dir, 'feeder_comparison.png'), dpi=300)
plt.show()

# 9. Time series decomposition for the largest consumer
# top_feeder = feeder_means.index[0]
# # Resample to daily data for clearer decomposition
# daily_series = plot_data.set_index('datetime')[top_feeder].resample('D').mean()
# # Fill missing values if any
# daily_series = daily_series.fillna(daily_series.mean())

# # Perform decomposition
# decomposition = seasonal_decompose(daily_series, model='additive', period=7)  # 7 days per week

# # Plot the decomposition components individually
# plt.figure(figsize=(12, 6))
# plt.plot(daily_series)
# plt.title(f'Original Time Series: {top_feeder}')
# plt.xlabel('Date')
# plt.ylabel('Energy (MWH)')
# plt.tight_layout()
# plt.savefig(os.path.join(plot_dir, 'decomp_original.png'), dpi=300)
# plt.show()

# plt.figure(figsize=(12, 6))
# plt.plot(decomposition.trend)
# plt.title(f'Trend Component: {top_feeder}')
# plt.xlabel('Date')
# plt.ylabel('Energy (MWH)')
# plt.tight_layout()
# plt.savefig(os.path.join(plot_dir, 'decomp_trend.png'), dpi=300)
# plt.show()

# plt.figure(figsize=(12, 6))
# plt.plot(decomposition.seasonal)
# plt.title(f'Seasonal Component: {top_feeder}')
# plt.xlabel('Date')
# plt.ylabel('Energy (MWH)')
# plt.tight_layout()
# plt.savefig(os.path.join(plot_dir, 'decomp_seasonal.png'), dpi=300)
# plt.show()

# plt.figure(figsize=(12, 6))
# plt.plot(decomposition.resid)
# plt.title(f'Residual Component: {top_feeder}')
# plt.xlabel('Date')
# plt.ylabel('Energy (MWH)')
# plt.tight_layout()
# plt.savefig(os.path.join(plot_dir, 'decomp_residual.png'), dpi=300)
#plt.show()

print(f"All plots have been saved to the '{plot_dir}' folder")

## Prepare Data for Hourly Time-Series Forecasting

In [ ]:
sequence_length = 24  # Use 24 hours of history for prediction
forecast_horizon = 24  # Predict the next 24 hours
batch_size = 32

data_model = data_clean.copy()
print(f"Data shape: {data_model.shape}")

In [ ]:
X_raw = data_model[time_feature_cols].values
y_raw = data_model[feeder_cols].values
print(f"X shape: {X_raw.shape}, y shape: {y_raw.shape}")

In [ ]:
feature_scaler = MinMaxScaler()
target_scaler = MinMaxScaler()

X_scaled = feature_scaler.fit_transform(X_raw)
y_scaled = target_scaler.fit_transform(y_raw)


In [ ]:
def create_sequences(X, y, seq_length, forecast_length):
    """Create input/output sequences for LSTM training from a dataframe"""
    X_seq, y_seq = [], []
    
    for i in range(len(X) - seq_length - forecast_length + 1):
        # Input sequence: previous seq_length time steps
        X_seq.append(X[i:i+seq_length])
        # Target sequence: next forecast_length time steps
        y_seq.append(y[i+seq_length:i+seq_length+forecast_length])
    
    return np.array(X_seq), np.array(y_seq)

In [ ]:
X_sequences, y_sequences = create_sequences(X_scaled, y_scaled, sequence_length, forecast_horizon)
print(f"Created sequences - X: {X_sequences.shape}, y: {y_sequences.shape}")


In [ ]:
# For time series, keep data in order (no shuffling)
X_train, X_test, y_train, y_test = train_test_split(
    X_sequences, y_sequences, test_size=0.2, shuffle=False, random_state=42
)

In [ ]:
print(f"Training data: X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"Testing data: X_test: {X_test.shape}, y_test: {y_test.shape}")


## Model Training 

### LSTM Training

In [ ]:
# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Get the dimensions from our training data
n_features = X_train.shape[2]  # Number of input features
n_outputs = y_train.shape[2]   # Number of target variables (feeders)

# Define the LSTM model
def create_lstm_model(sequence_length, n_features, n_outputs, forecast_horizon):
    model = models.Sequential([
        # Input layer
        layers.Input(shape=(sequence_length, n_features)),
        
        # LSTM layers with dropout to prevent overfitting
        layers.LSTM(128, return_sequences=True),
        layers.Dropout(0.2),
        
        layers.LSTM(64),
        layers.Dropout(0.2),
        
        # Dense layers to interpret LSTM outputs
        layers.Dense(64, activation='relu'),
        
        # Reshape to match the output dimensions
        layers.Dense(n_outputs * forecast_horizon),
        layers.Reshape((forecast_horizon, n_outputs))
    ])
    
    model.compile(
        optimizer='adam',
        loss='mse',  
        metrics=['mae', 'mape']
    )
    
    return model


In [ ]:
sequence_length

In [ ]:
# Create the model
model = create_lstm_model(
    sequence_length=sequence_length,
    n_features=n_features,
    n_outputs=n_outputs,
    forecast_horizon=forecast_horizon
)

In [ ]:
# Display model summary
model.summary()


In [ ]:
model_dir = 'models/'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

# Set up callbacks for training
callbacks_list = [
    callbacks.EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True
    ),
    callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=0.0001
    ),
    callbacks.ModelCheckpoint(
        filepath=os.path.join(model_dir, 'best_energy_forecast_model.keras'),
        monitor='val_loss',
        save_best_only=True
    ),
    callbacks.TensorBoard(
        log_dir='./logs',
        histogram_freq=1
    )
]

In [ ]:
print("Saving scalers to disk...")
joblib.dump(feature_scaler, os.path.join(model_dir, 'feature_scaler.pkl'))
joblib.dump(target_scaler, os.path.join(model_dir, 'target_scaler.pkl'))
print(f"Scalers saved to {model_dir}")

In [ ]:
# Train the model
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=batch_size,
    validation_split=0.2,  # Use 20% of training data as validation
    callbacks=callbacks_list,
    verbose=1
)

In [ ]:
# Evaluate on test set
test_loss, test_mae, test_mape = model.evaluate(X_test, y_test)
print(f"Test Loss (MSE): {test_loss:.4f}")
print(f"Test MAE: {test_mae:.4f}")
print(f"Test MAPE: {test_mape:.2f}%")

In [ ]:
# Plot training history
train_plot_dir = 'training_plots/'
if not os.path.exists(train_plot_dir):
    os.makedirs(train_plot_dir)

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss (MSE)')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')

plt.subplot(1, 2, 2)
plt.plot(history.history['mae'])
plt.plot(history.history['val_mae'])
plt.title('Model MAE')
plt.ylabel('MAE')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.tight_layout()
plt.savefig(os.path.join(train_plot_dir,'training_history.png'), dpi=300)
plt.show()

In [ ]:
# Save the final model
model.save(os.path.join(model_dir, 'lstm_model.keras'))

In [ ]:
# Make predictions on test data
y_pred = model.predict(X_test)

# Inverse transform the predictions and actual values to original scale
y_test_orig = target_scaler.inverse_transform(y_test.reshape(-1, n_outputs)).reshape(y_test.shape)
y_pred_orig = target_scaler.inverse_transform(y_pred.reshape(-1, n_outputs)).reshape(y_pred.shape)

# Calculate error metrics on original scale
mae = mean_absolute_error(y_test_orig.reshape(-1, n_outputs), y_pred_orig.reshape(-1, n_outputs))
mape = mean_absolute_percentage_error(y_test_orig.reshape(-1, n_outputs), y_pred_orig.reshape(-1, n_outputs))
rmse = root_mean_squared_error(y_test_orig.reshape(-1, n_outputs), y_pred_orig.reshape(-1, n_outputs))

print(f"Mean Absolute Error (in original scale): {mae:.4f}")
print(f"Mean Absolute Percentage Error: {mape*100:.2f}%")
print(f"Root Mean Squared Error (in original scale): {rmse:.4f}")


In [ ]:
pred_dir = 'prediction_plots/'
if not os.path.exists(pred_dir):
    os.makedirs(pred_dir)
# Plot predictions vs actual for the first test sample
sample_idx = 0

plt.figure(figsize=(15, 8))
# Plot for each feeder (first 4 for clarity)
for i in range(min(4, n_outputs)):
    plt.subplot(2, 2, i+1)
    plt.plot(y_test_orig[sample_idx, :, i], label='Actual')
    plt.plot(y_pred_orig[sample_idx, :, i], label='Predicted')
    plt.title(f'Feeder {feeder_cols[i]}')
    plt.xlabel('Hour')
    plt.ylabel('Energy (MWh)')
    plt.legend()

plt.tight_layout()
plt.savefig(os.path.join(pred_dir, 'sample_predictions.png'), dpi=300)
plt.show()

# Plot total energy consumption (sum of all feeders)
plt.figure(figsize=(12, 6))
plt.plot(np.sum(y_test_orig[sample_idx], axis=1), label='Actual Total')
plt.plot(np.sum(y_pred_orig[sample_idx], axis=1), label='Predicted Total')
plt.title('Total Energy Consumption - Actual vs Predicted')
plt.xlabel('Hour')
plt.ylabel('Total Energy (MWh)')
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(pred_dir, 'total_prediction.png'), dpi=300)
plt.show()

## SHAP Explainable AI

In [ ]:
# Create a section for SHAP Analysis
print("=" * 80)
print("SHAP ANALYSIS: EXPLAINING MODEL PREDICTIONS")
print("=" * 80)
timestep_explain= 0

def model_predict(X):
    # Convert to the shape expected by the LSTM (adding batch and sequence dimensions if needed)
    samples= X.shape[0]
    X_reshaped = X.reshape(samples, sequence_length, n_features)
    preds = model.predict(X_reshaped, verbose=1)
    return preds[:, timestep_explain, :]

background_data = 50  # Use 50 examples as background
background_samples = X_train[:background_data]

#flatten the background samples
background_samples = background_samples.reshape(background_data, -1)
print(f"Background data shape: {background_samples.shape}")

# Initialize the SHAP explainer with the model and background data
explainer = shap.KernelExplainer(model_predict, background_samples, link="identity")

# Choose a test sample to explain
sample = X_test[sample_idx:sample_idx+1]  # Recall sample_idx= 0
#print(f"Sample shape for SHAP: {sample.shape}")
sample_flat = sample.reshape(1, -1)
print(f"Sample shape for SHAP: {sample_flat.shape}")

# Calculate SHAP values for this sample
shap_values = explainer.shap_values(sample_flat)

# Print the shape of SHAP values to understand the structure
print(f"SHAP values shape: {np.array(shap_values).shape}")

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
# Create feature names for visualization
# We'll create feature names like 'hour_t0', 'day_of_week_t0', etc.
basic_feature_names = [
    "hour", "day_of_week", "month", "day", "year", "is_weekend",
    "hour_sin", "hour_cos", "month_sin", "month_cos",
    "day_of_week_sin", "day_of_week_cos"
]

feature_names = []
for t in range(sequence_length):
    for f in basic_feature_names:
        feature_names.append(f"{f}_t{t}")

# Plot summary for each feeder
plt.figure(figsize=(20, 12))
for i in range(min(4, n_outputs)):  # Just plot first 4 feeders
    plt.subplot(2, 2, i+1)


    if shap_values.shape[0] != sample_flat.shape[0]:
        raise ValueError(
            f"Mismatch for output {i}: SHAP values have shape {shap_values.shape} "
            f"but explain_samples have shape {sample_flat.shape}"
        )

    # For summary plot, we limit to top 20 features to keep it readable
    shap.summary_plot(
        shap_values,
        sample_flat,
        feature_names=feature_names,
        max_display=20,
        show=False
    )
    plt.title(f"Feature Importance for {feeder_cols[i]} Prediction")
    
plt.tight_layout()
plt.savefig(os.path.join(pred_dir, 'shap_feature_importance.png'), dpi=300)
plt.show()

# Create a more detailed visualization for the first feeder
plt.figure(figsize=(15, 8))

# Get the most important features for the first feeder
feature_importance = np.abs(shap_values[0].mean(0))
top_indices = np.argsort(-feature_importance)[:10]  # Top 10 features
top_names = [feature_names[i] for i in top_indices]

# Create a bar chart of the most important features
plt.barh(range(len(top_names)), [feature_importance[i] for i in top_indices])
plt.yticks(range(len(top_names)), top_names)
plt.title(f"Top 10 Important Features for {feeder_cols[0]}")
plt.xlabel("Mean |SHAP Value|")
plt.tight_layout()
plt.savefig(os.path.join(pred_dir, 'top_features.png'), dpi=300)
plt.show()

print("SHAP analysis completed. Results saved to the 'prediction_plots' directory.")

In [ ]:
# Create a mapping of feature indices to feature names for better interpretability
feature_names = [
    "hour", "day_of_week", "month", "day", "year", "is_weekend",
    "hour_sin", "hour_cos", "month_sin", "month_cos",
    "day_of_week_sin", "day_of_week_cos"
]

# Plot force plots for the first feeder to explain individual predictions in detail
plt.figure(figsize=(15, 8))
shap.initjs()  # Initialize JavaScript visualization

# Create a force plot for the first feeder
force_plot = shap.force_plot(
    explainer.expected_value[0],
    shap_values[0][0],
    sample_flat[0],
    feature_names=feature_names,
    matplotlib=True,
    show=False
)

plt.title(f"Detailed SHAP Explanation for {feeder_cols[0]} Prediction")
plt.savefig(os.path.join(pred_dir, 'shap_force_plot.png'), dpi=300)
plt.show()

# Create a dependence plot to show how a specific feature affects predictions
# For example, showing how hour_sin affects predictions for the first feeder
plt.figure(figsize=(12, 6))
shap.dependence_plot(
    "hour_sin", 
    shap_values[0], 
    sample_flat,
    feature_names=feature_names,
    show=False
)
plt.title(f"How Hour (Sine Transform) Affects {feeder_cols[0]} Predictions")
plt.savefig(os.path.join(pred_dir, 'shap_dependence_plot.png'), dpi=300)
plt.show()

# Time series specific analysis
# Create a plot showing SHAP values across time for specific features
# First, we need to generate SHAP values for a sequence of predictions
sequence_to_explain = X_test[sample_idx:sample_idx+1]
sequence_shap_values = explainer.shap_values(sequence_to_explain)

# Plot SHAP values for time-related features for the first feeder
time_feature_indices = [0, 6, 7]  # hour, hour_sin, hour_cos
time_feature_names = [feature_names[i] for i in time_feature_indices]

plt.figure(figsize=(15, 8))
for i, idx in enumerate(time_feature_indices):
    plt.subplot(len(time_feature_indices), 1, i+1)
    plt.bar(range(sequence_length), sequence_shap_values[0][0, idx])
    plt.title(f"SHAP Values for {time_feature_names[i]} Over Time")
    plt.ylabel("SHAP Value")
    plt.xlabel("Time Step")

plt.tight_layout()
plt.savefig(os.path.join(pred_dir, 'shap_time_features.png'), dpi=300)
plt.show()

print("SHAP analysis completed. Results saved to the 'prediction_plots' directory.")

## Feeder Allocation

In [ ]:
# Define transformer constraints
TRANSFORMERS = {
    'T1': {'capacity': 40, 'feeders': ['sango']},
    'T2': {'capacity': 60, 'feeders': ['fsm', 'amje', 'sumo']},
    'T3': {'capacity': 100, 'feeders': ['tower']},
    'T4': {'capacity': 60, 'feeders': ['qualitec', 'arrachid', 'ijagba', 'tollgate', 'aarti']},
    'T5': {'capacity': 40, 'feeders': ['idiroko', 'estate']}
}

In [ ]:
# Priority tiers
TIER1 = {
    'SECURITY': ['sango', 'tollgate']          # Security critical
}
TIER2 = {
    'HEALTHCARE': ['idiroko']                    # Healthcare critical                        
}
TIER3 = {
    'FINANCIAL': ['qualitec', 'aarti', 'tower', 'arrachid', 'sumo']  # Financial services are Dedicated Lines
}
TIER4 = {
    'GENERAL': ['ijagba', 'amje', 'estate']        # General services; Residential, Industrial and Commercial
}

# Flattened tiers for easier processing
PRIORITY_ORDER = [
    *TIER1['SECURITY'],    # Security critical (first)
    *TIER2['HEALTHCARE'],                         # Healthcare critical (second)
    *TIER3['FINANCIAL'],                        # Financial services (third)
    *TIER4['GENERAL']                            # General purpose (last)
]
PRIORITY_TIERS = [TIER1, TIER2, TIER3, TIER4]

In [ ]:
PRIORITY_TIERS

In [ ]:
PRIORITY_ORDER

In [ ]:
def allocate_energy(forecasted_demand, available_supply):
    """
    Allocate available energy supply to feeders based on priority tiers
    
    Parameters:
    -----------
    forecasted_demand : dict
        Dictionary mapping feeder names to their forecasted demand in MW
    available_supply : float
        Total available energy supply in MW
        
    Returns:
    --------
    dict
        Dictionary mapping feeder names to allocated energy in MW
    """
    total_demand = sum(forecasted_demand.values())
    allocation = {feeder: 0 for feeder in forecasted_demand}
    
    # Case 1: Supply exceeds or equals demand
    if available_supply >= total_demand:
        print(f"Supply ({available_supply:.2f} MW) exceeds demand ({total_demand:.2f} MW)")
        print("All feeders will receive 100% of their requested demand")
        return forecasted_demand
    
    # Case 2: Demand exceeds supply - allocate by priority
    print(f"Demand ({total_demand:.2f} MW) exceeds supply ({available_supply:.2f} MW)")
    print("Allocating based on priority tiers...")
    
    remaining_supply = available_supply
    
    # Allocate by priority order
    for feeder in PRIORITY_ORDER:
        # Skip if this feeder is not in our forecast
        if feeder not in forecasted_demand:
            continue
            
        feeder_demand = forecasted_demand[feeder]
        
        # If we have enough supply for this feeder
        if remaining_supply >= feeder_demand:
            allocation[feeder] = feeder_demand
            remaining_supply -= feeder_demand
            print(f"  Allocated {feeder_demand:.2f} MW to {feeder} (100% of demand)")
        
        # If we have some supply but not enough for full demand
        elif remaining_supply > 0:
            allocation[feeder] = remaining_supply
            print(f"  Allocated {remaining_supply:.2f} MW to {feeder} ({remaining_supply/feeder_demand*100:.1f}% of demand)")
            remaining_supply = 0
        
        # If we're out of supply
        else:
            print(f" No supply left for {feeder}")
    
    return allocation

In [ ]:
def analyze_allocation(allocation, forecasted_demand):
    """Analyze and report on the energy allocation"""
    total_allocated = sum(allocation.values())
    total_demand = sum(forecasted_demand.values())
    
    print("\nAllocation Analysis:")
    print(f"Total Demand: {total_demand:.2f} MW")
    print(f"Total Allocated: {total_allocated:.2f} MW")
    print(f"Allocation Rate: {total_allocated/total_demand*100:.1f}%")
    
    # Analyze by tier
    for tier_name, tier_dict in [("TIER1 (Critical)", TIER1), 
                               ("TIER2 (Financial)", TIER2),
                               ("TIER3 (Industrial)", TIER3), 
                               ("TIER4 (General)", TIER4)]:
        tier_feeders = [f for sublist in tier_dict.values() for f in sublist]
        tier_demand = sum(forecasted_demand.get(f, 0) for f in tier_feeders)
        tier_allocated = sum(allocation.get(f, 0) for f in tier_feeders)
        
        if tier_demand > 0:
            print(f"\n{tier_name}:")
            print(f"  Demand: {tier_demand:.2f} MW")
            print(f"  Allocated: {tier_allocated:.2f} MW")
            print(f"  Satisfaction Rate: {tier_allocated/tier_demand*100:.1f}%")
            
    # Check transformer constraints
    print("\nTransformer Loading Analysis:")
    for tx_name, tx_info in TRANSFORMERS.items():
        tx_capacity = tx_info['capacity']
        tx_feeders = tx_info['feeders']
        tx_allocation = sum(allocation.get(f, 0) for f in tx_feeders)
        tx_utilization = tx_allocation / tx_capacity * 100
        
        print(f"Transformer {tx_name} ({tx_capacity} MW capacity):")
        print(f"  Feeders: {', '.join(tx_feeders)}")
        print(f"  Allocated: {tx_allocation:.2f} MW")
        print(f"  Utilization: {tx_utilization:.1f}%")

In [ ]:
def generate_one_year_forecast(model, feature_scaler, target_scaler, initial_data, n_features, n_outputs, forecast_days=30):
    """
    Generate a one-year ahead forecast using the trained LSTM model
    
    Parameters:
    -----------
    model : keras.Model
        Trained LSTM model
    feature_scaler : MinMaxScaler
        Scaler used for features
    target_scaler : MinMaxScaler
        Scaler used for targets
    initial_data : np.array
        Initial input data for the model (must be of shape [1, sequence_length, n_features])
    n_features : int
        Number of input features
    n_outputs : int
        Number of outputs (number of feeders)
    forecast_days : int
        Number of days to forecast (default: 365)
        
    Returns:
    --------
    np.array
        Forecasted values for each feeder for each day in original scale
    """
    # Initialize containers for forecast
    forecast_hours = forecast_days * 24  # 24 hours per day
    predictions = np.zeros((forecast_hours, n_outputs))
    
    # Make a copy of the initial input data
    current_input = initial_data.copy()
    
    # Generate forecasts one day at a time
    for hour in range(forecast_hours):
        # Generate forecast for next hour
        next_hour_forecast = model.predict(current_input)
        
        # Store the forecast for this hour
        predictions[hour] = next_hour_forecast[0, 0, :]
        
        # Update the input for the next prediction
        # Shift the window forward by one step and append the new prediction
        current_input[0, :-1, :] = current_input[0, 1:, :]
        
        # Create new features for the next time step (this would need customization based on your features)
        # This is a simplified version that assumes your features include hour of day, day of week, etc.
        # For a real implementation, you'd need to update all time-based features correctly
        # For now, we'll just use the last row's features
        current_input[0, -1, :] = current_input[0, -1, :]
    
    # Convert predictions back to original scale
    predictions_orig = target_scaler.inverse_transform(predictions)
    
    return predictions_orig



In [ ]:
# Function to simulate the allocation system with different supply scenarios
def run_allocation_simulation(forecasted_demand, base_demand_total, variation_percentages=[-20, -10, 0, 10, 20]):
    """
    Run energy allocation simulations for different supply scenarios
    
    Parameters:
    -----------
    forecasted_demand : dict
        Dictionary mapping feeder names to their forecasted demand in MW
    base_demand_total : float
        Total base demand for normalization
    variation_percentages : list
        List of percentage variations from the base demand to simulate
    """
    print("=" * 80)
    print("ENERGY ALLOCATION SIMULATION")
    print("=" * 80)
    
    # Test with different supply levels
    for variation in variation_percentages:
        available_supply = base_demand_total * (1 + variation/100)
        print("\n" + "=" * 60)
        print(f"SCENARIO: Supply at {100+variation}% of demand ({available_supply:.2f} MW)")
        print("=" * 60)
        
        # Run the allocation algorithm
        allocation = allocate_energy(forecasted_demand, available_supply)
        
        # Analyze the results
        analyze_allocation(allocation, forecasted_demand)

In [ ]:
def visualize_allocation(allocation, forecasted_demand):
    """
    Visualize the energy allocation results
    
    Parameters:
    -----------
    allocation : dict
        Dictionary mapping feeder names to allocated energy in MW
    forecasted_demand : dict
        Dictionary mapping feeder names to forecasted demand in MW
    """
    # Create allocation plot directory
    alloc_dir = 'allocation_plots/'
    if not os.path.exists(alloc_dir):
        os.makedirs(alloc_dir)
    
    # Prepare data for visualization
    feeders = list(allocation.keys())
    allocated = [allocation[f] for f in feeders]
    demanded = [forecasted_demand[f] for f in feeders]
    
    # Calculate allocation percentage
    allocation_pct = [allocation[f]/forecasted_demand[f]*100 if forecasted_demand[f] > 0 else 0 for f in feeders]
    
    # Sort by priority tier
    tier_order = {f: i for i, f in enumerate(PRIORITY_ORDER)}
    feeders_sorted = sorted(feeders, key=lambda x: tier_order.get(x, 999))
    allocated_sorted = [allocation[f] for f in feeders_sorted]
    demanded_sorted = [forecasted_demand[f] for f in feeders_sorted]
    allocation_pct_sorted = [allocation[f]/forecasted_demand[f]*100 if forecasted_demand[f] > 0 else 0 for f in feeders_sorted]
    
    # Create a color map for the tiers
    tier_colors = {
        'SECURITY': 'darkred',
        'HEALTHCARE': 'red',
        'FINANCIAL': 'orange',
        'INDUSTRIAL': 'teal',
        'GENERAL': 'gray'
    }
    
    colors = []
    for f in feeders_sorted:
        if f in TIER1['SECURITY']:
            colors.append(tier_colors['SECURITY'])
        elif f in TIER1['HEALTHCARE']:
            colors.append(tier_colors['HEALTHCARE'])
        elif f in TIER2['FINANCIAL']:
            colors.append(tier_colors['FINANCIAL'])
        elif f in TIER3['INDUSTRIAL']:
            colors.append(tier_colors['INDUSTRIAL'])
        else:
            colors.append(tier_colors['GENERAL'])
    
    # 1. Allocation vs Demand Bar Chart
    plt.figure(figsize=(15, 8))
    x = range(len(feeders_sorted))
    width = 0.35
    
    plt.bar(x, demanded_sorted, width, label='Demand', color='lightgray', edgecolor='black')
    plt.bar(x, allocated_sorted, width, label='Allocated', color=colors, alpha=0.7)
    
    plt.xlabel('Feeder')
    plt.ylabel('Energy (MW)')
    plt.title('Energy Allocation vs Demand by Feeder')
    plt.xticks(x, feeders_sorted, rotation=45)
    plt.legend()
    
    # Add tier-based labels
    plt.axvline(x=len(TIER1['SECURITY'])+len(TIER1['HEALTHCARE'])-0.5, color='black', linestyle='--')
    plt.axvline(x=len(TIER1['SECURITY'])+len(TIER1['HEALTHCARE'])+len(TIER2['FINANCIAL'])-0.5, color='black', linestyle='--')
    plt.axvline(x=len(TIER1['SECURITY'])+len(TIER1['HEALTHCARE'])+len(TIER2['FINANCIAL'])+len(TIER3['INDUSTRIAL'])-0.5, color='black', linestyle='--')
    
    plt.tight_layout()
    plt.savefig(os.path.join(alloc_dir, 'allocation_vs_demand.png'), dpi=300)
    plt.show()
    
    # 2. Allocation Percentage Bar Chart
    plt.figure(figsize=(15, 8))
    plt.bar(x, allocation_pct_sorted, color=colors, alpha=0.7)
    plt.axhline(y=100, color='red', linestyle='--')
    
    plt.xlabel('Feeder')
    plt.ylabel('Allocation Percentage (%)')
    plt.title('Energy Allocation Percentage by Feeder')
    plt.xticks(x, feeders_sorted, rotation=45)
    
    # Add tier-based labels
    plt.axvline(x=len(TIER1['SECURITY'])+len(TIER1['HEALTHCARE'])-0.5, color='black', linestyle='--')
    plt.axvline(x=len(TIER1['SECURITY'])+len(TIER1['HEALTHCARE'])+len(TIER2['FINANCIAL'])-0.5, color='black', linestyle='--')
    plt.axvline(x=len(TIER1['SECURITY'])+len(TIER1['HEALTHCARE'])+len(TIER2['FINANCIAL'])+len(TIER3['INDUSTRIAL'])-0.5, color='black', linestyle='--')
    
    plt.tight_layout()
    plt.savefig(os.path.join(alloc_dir, 'allocation_percentage.png'), dpi=300)
    plt.show()
    
    # 3. Tier-based allocation pie chart
    plt.figure(figsize=(12, 8))
    tier_names = ['Security', 'Healthcare', 'Financial', 'Industrial', 'General']
    tier_demands = [
        sum(forecasted_demand.get(f, 0) for f in TIER1['SECURITY']),
        sum(forecasted_demand.get(f, 0) for f in TIER1['HEALTHCARE']),
        sum(forecasted_demand.get(f, 0) for f in TIER2['FINANCIAL']),
        sum(forecasted_demand.get(f, 0) for f in TIER3['INDUSTRIAL']),
        sum(forecasted_demand.get(f, 0) for f in TIER4['GENERAL'])
    ]
    tier_allocated = [
        sum(allocation.get(f, 0) for f in TIER1['SECURITY']),
        sum(allocation.get(f, 0) for f in TIER1['HEALTHCARE']),
        sum(allocation.get(f, 0) for f in TIER2['FINANCIAL']),
        sum(allocation.get(f, 0) for f in TIER3['INDUSTRIAL']),
        sum(allocation.get(f, 0) for f in TIER4['GENERAL'])
    ]
    tier_colors_list = [tier_colors['SECURITY'], tier_colors['HEALTHCARE'], tier_colors['FINANCIAL'], tier_colors['INDUSTRIAL'], tier_colors['GENERAL']]
    
    plt.subplot(1, 2, 1)
    plt.pie(tier_demands, labels=tier_names, autopct='%1.1f%%', colors=tier_colors_list)
    plt.title('Demand Distribution by Priority Tier')
    
    plt.subplot(1, 2, 2)
    plt.pie(tier_allocated, labels=tier_names, autopct='%1.1f%%', colors=tier_colors_list)
    plt.title('Allocation Distribution by Priority Tier')
    
    plt.tight_layout()
    plt.savefig(os.path.join(alloc_dir, 'tier_allocation.png'), dpi=300)
    plt.show()

In [ ]:
model

In [ ]:
def run_intelligent_energy_management(model, feature_scaler, target_scaler, last_known_data, feeder_cols, supply_variations=[-20, -10, 0, 10, 20]):
    """
    Run the intelligent energy management system
    
    Parameters:
    -----------
    model : keras.Model
        Trained LSTM model
    feature_scaler : MinMaxScaler
        Scaler used for features
    target_scaler : MinMaxScaler
        Scaler used for targets
    last_known_data : np.array
        Last known data point to start the forecast
    feeder_cols : list
        List of feeder column names
    supply_variations : list
        List of percentage variations from the base demand to simulate
    """
    # Generate forecast for next 24 hours
    # For a full year forecast, you would use generate_one_year_forecast() instead
    next_day_forecast = model.predict(last_known_data)
    
    # Convert forecast to original scale
    n_outputs = len(feeder_cols)
    forecast_orig = target_scaler.inverse_transform(
        next_day_forecast.reshape(-1, n_outputs)
    ).reshape(next_day_forecast.shape)
    
    # Calculate total energy demand for each feeder for the next day
    daily_demand = {}
    for i, feeder in enumerate(feeder_cols):
        # Sum across 24 hours to get total daily demand
        daily_demand[feeder] = np.sum(forecast_orig[0, :, i])
    
    # Run the allocation simulation with different supply scenarios
    total_demand = sum(daily_demand.values())
    sharing_ratios = {feeder: (demand/total_demand*100) for feeder, demand in daily_demand.items()}

# Print the sharing ratios in descending order
    for feeder, ratio in sorted(sharing_ratios.items(), key=lambda x: x[1], reverse=True):
        print(f"The sharing ratio for {feeder}: {ratio:.2f}%")
    run_allocation_simulation(daily_demand, total_demand, supply_variations)
    
    # Store allocations for each scenario
    allocations = {}
    for variation in supply_variations:
        available_supply = total_demand * (1 + variation/100)
        scenario_name = f"{100+variation}% of demand ({available_supply:.2f} MW)"
        allocations[scenario_name] = allocate_energy(daily_demand, available_supply)
    
    # Visualize each scenario (or select specific ones to visualize)
    for scenario_name, allocation in allocations.items():
        print(f"\nVisualizing scenario: {scenario_name}")
        visualize_allocation(allocation, daily_demand)
    
    # Return the demand and all allocations
    return daily_demand, allocations

In [ ]:
# Example usage
# Assuming you have the last test sample as last_known_data
last_known_data = X_test[[5]]  # First test sample

# Run the energy management system
daily_demand, allocation = run_intelligent_energy_management(
    model,
    feature_scaler,
    target_scaler,
    last_known_data,
    feeder_cols,
    supply_variations=[-20, -10, 0, 10, 20]
)